In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import functools
from traitlets import traitlets

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn import tree
import plotly.express as px

# Declaring Functions

## Model Inits

In [ ]:
def init_nn(activation='tanh', alpha=0.001, l1_size=8, l2_size=4, l3_size=0, l4_size=0, lr=0.001):
    h_layer_sizes = (l1_size,)
    
    if l2_size != 0 and l3_size == 0 and l4_size == 0:
        h_layer_sizes = (l1_size, l2_size)
    elif l2_size != 0 and l3_size != 0 and l4_size == 0:
        h_layer_sizes = (l1_size, l2_size, l3_size)
    elif l2_size != 0 and l3_size != 0 and l4_size != 0:
        h_layer_sizes = (l1_size, l2_size, l3_size, l4_size)
            
    clf = MLPClassifier(activation=activation, solver='adam', alpha=alpha, hidden_layer_sizes=h_layer_sizes,
                    learning_rate="constant", learning_rate_init=lr,
                    random_state=42, max_iter=1, warm_start=True)
    return clf

## Dataset Inits

In [ ]:
def init_nn_dataset(dataset="Circular"):
    if dataset == "Circular":
        df = pd.read_csv("classification_circle.csv")
        df.loc[ df["y"] == -1, "y"] = 0 
    elif dataset == "XOR":
        df = pd.read_csv("classification_nonlinear_xor.csv")
        df.loc[ df["y"] == -1, "y"] = 0
    
    feature_cols = ['x1', 'x2']
    X = df[feature_cols].to_numpy()
    y = df['y'].to_numpy()
    
    return X, y, df

## Animations

In [ ]:
def animate_nn(i, *fargs):
    clf, X, y, df, xx, yy = fargs
    xmin, xmax = df["x1"].min(), df["x1"].max() 
    ymin, ymax = df["x2"].min(), df["x2"].max()
    clf.fit(X, y)
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.cla()
    plt.imshow(
    Z,
    interpolation="nearest",
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    aspect="auto",
    origin="lower",
    alpha = 0.7,
    cmap=plt.cm.Paired
    )
    contours = plt.contour(xx, yy, Z, linewidths=2, colors="white")
    sns.scatterplot(x="x1", y="x2", hue="y", data=df)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    plt.legend(loc='lower right')

## Plays

In [ ]:
def play_nn(btn_state, activation='tanh', alpha=0.001, l1_size=8, l2_size=4, l3_size=0,
            l4_size=0, lr=0.001, dataset='Circular'):
    
    btn_state.disabled = True
    
    anim_running = True
    def onClick(event): # start/stop animation when clicked on figure 
        nonlocal anim_running
        if anim_running:
            anim.event_source.stop()
            anim_running = False
        else:
            anim.event_source.start()
            anim_running = True
    
    plt.clf() # clear figure  
    clf = init_nn(activation=activation, alpha=alpha, l1_size=l1_size, l2_size=l2_size, l3_size=l3_size,
                  l4_size=l4_size, lr=lr)    
    X, y, df = init_nn_dataset(dataset=dataset)
    
    xmin, xmax = df["x1"].min(), df["x1"].max() 
    ymin, ymax = df["x2"].min(), df["x2"].max()
    xx, yy = np.meshgrid(np.linspace(xmin, xmax, 500), np.linspace(xmin, xmax, 500))
    plt.gcf().canvas.mpl_connect('button_press_event', onClick) # connects the figure with the onClick func
    anim = FuncAnimation(plt.gcf(), func=animate_nn, fargs=(clf, X, y, df, xx, yy), interval=100,
                         frames=200, blit=True)
    btn_state.value = anim
    return anim

## Tests

In [ ]:
def test(btn_state, activation='tanh', alpha=0.001, l1_size=8, l2_size=4, l3_size=0,
            l4_size=0, lr=0.001, dataset='Circular'):
    print(activation, alpha, l1_size, l2_size, l3_size, l4_size, lr, dataset)

In [ ]:
def init_logreg(a, c, d):
    return a, c, d

def animate_logreg(a, c, d):
    print(a, c, d)

def on_button_clicked_2(b, a, c, d):
    with output:
        e, f, g = init_logreg(a, c, d)
        animate_logreg(e, f, g)

# Declaring Widgets

In [ ]:
class LoadedButton(widgets.Button):
    """A custom button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

In [ ]:
algorithms = ['Linear Regression', 'Logistic Regression', 'Neural Network', 'Linear SVM', 'Non-linear SVM', 'KMeans',
             'Naive Bayes', 'Decision Tree', 'PCA', 'None']

output = widgets.Output()

algo_dropdown = widgets.Dropdown(
            options=algorithms,
            value='None',
            description='Algorithm:',
            disabled=False,
        )

activation_dropdown = widgets.Dropdown(
            options=['logistic', 'tanh', 'relu'],
            value='tanh',
            description='Activation:',
            disabled=False,
        )

lr_slider = widgets.FloatSlider(
            value=0.001,
            min=0.0001,
            max=0.1,
            step=0.0001,
            description='alpha:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.4f',
        )

reg_strength_slider = widgets.FloatSlider(
            value=0.001,
            min=0.0001,
            max=0.1,
            step=0.0001,
            description='lambda:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.4f',
        )

regtype_dropdown = widgets.Dropdown(
            options=['L1', 'L2', 'None'],
            value='L2',
            description='Reg:',
            disabled=False,
        )

l1_dropdown = widgets.Dropdown(
            options=[1, 2, 4, 8],
            value=8,
            description='L1 Size:',
            disabled=False,
        )

l2_dropdown = widgets.Dropdown(
            options=[0, 1, 2, 4, 8],
            value=4,
            description='L2 Size:',
            disabled=False,
        )

l3_dropdown = widgets.Dropdown(
            options=[0, 1, 2, 4, 8],
            value=0,
            description='L3 Size:',
            disabled=False,
        )

l4_dropdown = widgets.Dropdown(
            options=[0, 1, 2, 4, 8],
            value=0,
            description='L4 Size:',
            disabled=False,
        )

dataset_nn_dropdown = widgets.Dropdown(
            options=['Circular', 'XOR'],
            value='Circular',
            description='Dataset',
            disabled=False,
        )

play_nn_btn = LoadedButton(icon="play")
play_lr_btn = LoadedButton(icon="play")

# Main 

In [ ]:
%matplotlib notebook
%matplotlib notebook
def run(algo='None'):
    if algo == "Logistic Regression":
        display(lr_slider, regtype_dropdown, reg_strength_slider, play_lr_btn, output)
        play_lr_btn.on_click(functools.partial(on_button_clicked_2, a=lr_slider.value, c=regtype_dropdown.value,
                                          d=reg_strength_slider.value))
        
    elif algo == "Neural Network":
        play_nn_btn.disabled = False
        hb1 = widgets.HBox([lr_slider, reg_strength_slider])
        hb2 = widgets.HBox([l1_dropdown, l2_dropdown, l3_dropdown, l4_dropdown])
        vb = widgets.VBox([activation_dropdown, hb1, hb2, dataset_nn_dropdown, play_nn_btn, output])
        display(vb)
        play_nn_btn.on_click(functools.partial(play_nn, activation=activation_dropdown.value,
                                            alpha=reg_strength_slider.value,
                                            l1_size=l1_dropdown.value,
                                            l2_size=l2_dropdown.value, l3_size=l3_dropdown.value,
                                            l4_size=l4_dropdown.value, lr=lr_slider.value,
                                            dataset=dataset_nn_dropdown.value))
        return play_nn_btn.value

In [ ]:
widgets.interact(run, algo=algo_dropdown);